<a href="https://colab.research.google.com/github/CateMerfeld/class_work/blob/main/Copy_of_Day_80_Lecture_1_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Long Short-Term Memory 

In this assignment, we will learn about LSTM models. We will create an LSTM model for time series prediction.

In [1]:
import numpy as np
import os
import pandas as pd

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout, Flatten
from tensorflow.keras.utils import to_categorical

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Below is a function for loading time series data collected by sensors. There are 9 different files, We have data about body acceleration, body gyro, and total acceleration for the x, y, and z axis

In [3]:
def load_func(path, file_ind=False):
    data_list = []
    if file_ind:
        filenames = [path]
    else:
        files = os.listdir(path)
        filenames = [path + '/' + f for f in files]
    for f in filenames:
        dataframe = pd.read_csv(f, header=None, delim_whitespace=True)
        data_list.append(dataframe.values)
    if len(data_list) > 1:
        return np.dstack(data_list)
    else:
        return data_list[0]

In [4]:
os.listdir('/content/drive/MyDrive/Data_Science_Course/Machine Learning/deep learning/UCI_HAR_Dataset/UCI HAR Dataset/train/Inertial Signals/')

['body_acc_x_train.txt',
 'body_gyro_z_train.txt',
 'total_acc_z_train.txt',
 'body_acc_y_train.txt',
 'body_acc_z_train.txt',
 'body_gyro_y_train.txt',
 'total_acc_y_train.txt',
 'total_acc_x_train.txt',
 'body_gyro_x_train.txt']

In [5]:
X_train = load_func("/content/drive/MyDrive/Data_Science_Course/Machine Learning/deep learning/UCI_HAR_Dataset/UCI HAR Dataset/train/Inertial Signals")
X_test = load_func("/content/drive/MyDrive/Data_Science_Course/Machine Learning/deep learning/UCI_HAR_Dataset/UCI HAR Dataset/test/Inertial Signals")
y_train_cat = load_func('/content/drive/MyDrive/Data_Science_Course/Machine Learning/deep learning/UCI_HAR_Dataset/UCI HAR Dataset/train/y_train.txt', True)
y_test_cat = load_func('/content/drive/MyDrive/Data_Science_Course/Machine Learning/deep learning/UCI_HAR_Dataset/UCI HAR Dataset/test/y_test.txt', True)

Print the dimensions of both the predictor variables and the target.

In [6]:
# Answer below:
print('Training vars shape: ', X_train.shape, 'Training target shape: ', y_train_cat.shape)


Training vars shape:  (7352, 128, 9) Training target shape:  (7352, 1)


The target variable is categorical. One hot encode the target variable.

In [7]:
y_train = pd.DataFrame(y_train_cat)
y_test = pd.DataFrame(y_test_cat)
y_train.nunique()

0    6
dtype: int64

In [8]:
# Answer below:
y_train = to_categorical(y_train_cat - 1, 6)
# pd.get_dummies(y_train_cat.flatten())
y_test = to_categorical(y_test_cat -1, 6)
# pd.get_dummies(y_test_cat.flatten())

Create a model containing an LSTM layer with unit size 100, and input shape that is the tuple containing the number of columns in X and the number of files in X.

The next layer is a dropout layer. Choose 0.5 for the proportion. Then add a dense layer of unit size 100 and finally an output dense layer. 

In [12]:
# Answer below:
model = Sequential()

model.add(LSTM(100, input_shape=X_train.shape[1:3]))
model.add(Dropout(0.5))
model.add(Dense(100))
model.add(Dense(6, activation='softmax'))

Print the model summary to ensure you have the correct number of parameters.

In [13]:
# Answer below:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 100)               44000     
_________________________________________________________________
dropout_1 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_3 (Dense)              (None, 6)                 606       
Total params: 54,706
Trainable params: 54,706
Non-trainable params: 0
_________________________________________________________________


Compile and fit the model. Select the appropriate activation, loss, and optimizer.

Run the model for 10 epochs with a batch size of 80.

In [14]:
# Answer below:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.fit(X_train, y_train, validation_data=(X_test, y_test),
          batch_size=80, epochs=10)

Epoch 1/10
92/92 [==============================] - 9s 17ms/step - loss: 1.3514 - accuracy: 0.4328 - val_loss: 3.1462 - val_accuracy: 0.1524
Epoch 2/10
92/92 [==============================] - 1s 11ms/step - loss: 0.7062 - accuracy: 0.6941 - val_loss: 4.6054 - val_accuracy: 0.1035
Epoch 3/10
92/92 [==============================] - 1s 11ms/step - loss: 0.4801 - accuracy: 0.8105 - val_loss: 5.3668 - val_accuracy: 0.1439
Epoch 4/10
92/92 [==============================] - 1s 11ms/step - loss: 0.3414 - accuracy: 0.8944 - val_loss: 5.4368 - val_accuracy: 0.0794
Epoch 5/10
92/92 [==============================] - 1s 11ms/step - loss: 0.1912 - accuracy: 0.9305 - val_loss: 7.3926 - val_accuracy: 0.0814
Epoch 6/10
92/92 [==============================] - 1s 11ms/step - loss: 0.2353 - accuracy: 0.9084 - val_loss: 6.0743 - val_accuracy: 0.0970
Epoch 7/10
92/92 [==============================] - 1s 11ms/step - loss: 0.1791 - accuracy: 0.9338 - val_loss: 6.8744 - val_accuracy: 0.1127
Epoch 8/10
92

Print the confusion matrix for the test data.

In [ ]:
# Answer below:
y_preds = model.predict_classes(X_test)
#np.argmax(model.predict(x), axis=-1)


In [ ]:
test_preds.shape

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns

y_preds = model.predict_classes(X_test)

cm = confusion_matrix(y_test_cat-1, y_preds)

sns.heatmap(cm, annot=True)